In [7]:

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate,MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough


"""
messageplaceholeder안에 history 메세지값을 넣어주길 원한다(수와숫자 상관없음)
수동으로 저장하고 가져오기
쳇프롬프트 템플릿으로 저장하는데 messageplaceholeder 사용한다 휴먼은 변수로 설정
load_memory를 써서 예전 데이터를 가져오는 함수를 만든다.
체인에 (데이터전달)RunnablePassthrough.assign 써서 역는다 | prompt | llm
invoke_chain 함수 만들기 result = invoke 설정하고 , 중간에 memory.save 당겨와서내용 저장, 마지막 print 결과 보여준다.
"""
llm= ChatOpenAI(temperature=0.1)



memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="history",
    return_messages= True,
    )

prompt= ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI taking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human","{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [8]:
invoke_chain("My name is 성호")

content='Hello 성호! How can I assist you today?'


In [9]:
invoke_chain("What is my name?")

content='Your name is 성호.'
